In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkContext

In [3]:
c = SparkContext.getOrCreate()
spark = SparkSession.builder \
    .appName("Cleaning data from review csv") \
    .master("local[*]") \
    .getOrCreate()
    


In [4]:
review_df = spark.read.csv("C:/Users/macmi/Desktop/Prj/ELT-prj/files/data_review.csv", header=True, inferSchema=True)

In [5]:
products = spark.read.csv("C:/Users/macmi/Desktop/Prj/ELT-prj/files/products.csv", header=True, inferSchema=True)
clients = spark.read.csv("C:/Users/macmi/Desktop/Prj/ELT-prj/files/clients.csv", header=True, inferSchema=True)

In [6]:
products.printSchema()
clients.printSchema()


root
 |-- product_id: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)

root
 |-- client_id: string (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)
 |-- first_redeem_date: timestamp (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)



In [7]:
# lấy các giá trị unique và nhét vào một bảng
# customer_id = products['product_id'].unique()   Cách này sai
products_id = products.select('product_id').distinct().collect() #collect để lấy về
print(type(products_id))
print(products_id[0])

<class 'list'>
Row(product_id='01e0bab3d9')


In [8]:
products_id_list = [row['product_id'] for row in products_id]
print(products_id_list[0])

01e0bab3d9


In [9]:
clients_id = clients.select('client_id').distinct().collect() #collect để lấy về
print(type(clients_id))
print(clients_id[0])

<class 'list'>
Row(client_id='001ecff0a8')


In [10]:
clients_id_list = [row['client_id'] for row in clients_id]
print(clients_id_list[0])

001ecff0a8


In [11]:
review_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [12]:
import random
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [13]:
# Cách làm này lỗi
random_product_udf = udf(lambda: random.choice(products_id_list), StringType())
random_user_udf = udf(lambda: random.choice(clients_id_list), StringType())

In [14]:
# cũng lỗi
review_df_randomized = review_df \
    .withColumn("ProductId", random_product_udf()) \
    .withColumn("UserId", random_user_udf())

# Xem thử kết quả
review_df_randomized.show(5)

Py4JJavaError: An error occurred while calling o92.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 32) (DESKTOP-26JB1TS executor driver): java.net.SocketException: Connection reset by peer
	at java.base/sun.nio.ch.SocketDispatcher.write0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.write(SocketDispatcher.java:54)
	at java.base/sun.nio.ch.NioSocketImpl.tryWrite(NioSocketImpl.java:394)
	at java.base/sun.nio.ch.NioSocketImpl.implWrite(NioSocketImpl.java:413)
	at java.base/sun.nio.ch.NioSocketImpl.write(NioSocketImpl.java:440)
	at java.base/sun.nio.ch.NioSocketImpl$2.write(NioSocketImpl.java:819)
	at java.base/java.net.Socket$SocketOutputStream.write(Socket.java:1195)
	at java.base/java.io.BufferedOutputStream.implWrite(BufferedOutputStream.java:217)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:200)
	at java.base/jav

In [15]:
from pyspark.sql.functions import array, lit, rand, floor, size, element_at

In [16]:
from pyspark.sql import Row
from pyspark.sql.functions import rand

# Chuyển list thành DataFrame
products_df = spark.createDataFrame([Row(product_id=x) for x in products_id_list])
clients_df = spark.createDataFrame([Row(user_id=x) for x in clients_id_list])

In [ ]:
# not runnable
random_product = products_id_df.orderBy(rand()).limit(1)
random_client = clients_df.orderBy(rand()).limit(1)

In [17]:
spark.conf.set("spark.sql.crossJoin.enabled", True)
spark.conf.set("spark.hadoop.fs.file.impl.disable.cache", True)


In [18]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import random

random_product_udf = udf(lambda: random.choice(products_id_list), StringType())
random_client_udf = udf(lambda: random.choice(clients_id_list), StringType())

review_df_randomized = review_df \
    .withColumn("ProductId", random_product_udf()) \
    .withColumn("UserId", random_client_udf())


In [19]:
import os
os.environ["HADOOP_HOME"]="C:/Users/macm/Downloads/hadoop/winutils/hadoop-3.3.6"

review_df_randomized.coalesce(1).write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("output/review_randomized")


Py4JJavaError: An error occurred while calling o166.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 33) (DESKTOP-26JB1TS executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/c:/Users/macmi/Desktop/Prj/ELT-prj/files/output/review_randomized.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:775)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at o

Py4JJavaError: An error occurred while calling o98.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6